In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from transfer import ObjectStyleTransfer

In [ ]:
trans = ObjectStyleTransfer('fcn8s_pascal_best_model.pkl')

In [ ]:
img = trans.generate(
    content_img='output/content.jpg',
    style_img='output/style.jpg')

In [ ]:
import torchvision.transforms as transforms

In [ ]:
transforms.ToPILImage()(img)